In [ ]:
import requests
import time
from geopy.distance import geodesic
import datetime
import geocoder
import csv

stationDict = {}
station2016Dict={}
stationJson = requests.get('https://gbfs.citibikenyc.com/gbfs/es/station_information.json').json()
zipcodeDensity = {}
crimeLocList = []
crimePerZipDict = {}
zipList = set()


In [ ]:
#put bike stations and location into local dict and csv file
def createFullStationCSV():
    with open('bikeShareData.csv', 'w', newline='') as f:
        fieldnames = ['id', 'name', 'lat', 'lon']
        thewriter = csv.DictWriter(f, fieldnames = fieldnames)
        thewriter.writeheader()

        for each in stationJson['data']['stations']:
            #print(each['name'])
            id = each['station_id']
            thewriter.writerow({'id' : id, 'name' : each['name'], 'lat' : each['lat'], 'lon' :  each['lon']})
            stationDict[id] = [each['name'], each['lat'], each['lon']]

In [ ]:
createFullStationCSV()

In [ ]:
def create2016StationCSV(fileName):
    count = 1
    with open(fileName) as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        next(readCSV)
        for row in readCSV:
            #print(row[0], row[5], row[6])
            #print(valueS)
            lat = row[5]
            lon =  row[6]
            station2016Dict[row[3]] = [row[3],row[5],row[6],0]
            
        print("Total stations read: ",len(station2016Dict))
            
    interval = len(station2016Dict)/20
    baseInterval = interval
    progress = 5
    with open('2016Stations.csv', 'w', newline='') as f:
        fieldnames = ['id', 'lat', 'lon', 'zip']
        print("geocoding...")
        thewriter = csv.writer(f)
        thewriter.writerow(fieldnames)
        print("progress: ", end="")
        for key, value in station2016Dict.items():
            g = geocoder.arcgis([value[1], value[2]], method='reverse')
            value[3] = g.postal
            zipList.add(value[3])
            if count > interval:
                print(progress,"%", end=" ")
                progress +=5
                interval += baseInterval
            thewriter.writerow(value)
            time.sleep(.01)
            count+=1
        print(progress,"%", end=" ")
        print()
        #print("Zipcode Density", len(zipcodeDensity))
        #print(zipcodeDensity)

In [ ]:
create2016StationCSV("201612-citibike-tripdata.csv")

Dec 2016 and other trip datasets can be found here https://s3.amazonaws.com/tripdata/index.html
There are still pieces of missing zipcodes for some of the bike docks. These will need to be manually filled in via Google Maps. Geocoder does support googlemaps, but a seperate API key is needed.

In [ ]:
def bikeDensityForZip(fileName):
    count = 1
    with open(fileName) as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        #skips the header
        next(readCSV)
        for row in readCSV:
            zip = row[3]
            if not zip in zipcodeDensity:
                zipcodeDensity[zip] = 1
            else:
                zipcodeDensity[zip] +=1            

In [ ]:
bikeDensityForZip("2016Stations.csv")

In [ ]:
def loadFelonies(fileName):
    with open(fileName) as csvfile:
        readCSV = csv.reader(csvfile)
        for row in readCSV:
            #List format: Crime Type, Lat, Long, Zip
            if row[12] == "FELONY":
                crimeLocList.append([row[12], row[22], row[23], 0])
    print("Felony Count: ", len(crimeLocList))

In [ ]:
loadFelonies('crime_2016.csv')

In [ ]:
def revGeoCodeFelonies():
    with open('crimewithzip.csv', 'w') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["type", "lat", "long", 'zip'])
        t0 = time.time()
        print(t0)
        rowcounter = 0
        for row in crimeLocList:
            g = geocoder.arcgis([row[1], row[2]], method='reverse')
            zipcode = g.postal
            #update crimeLocList
            print(zipcode)
            row[3] = zipcode
            print(row[0],row[1],row[2],row[3])
            #write to csv
            writer.writerow(row)
            t1 = time.time() - t0
            if(t1 > 60):
                print(t1)
            time.sleep(.1)
        rowcounter += 1
        print(rowcounter)

Note: running revGeoCodeFelonies() can take 40+ hours as there is over 140k entries.

In [ ]:
revGeoCodeFelonies()

In [ ]:
def crimePerZipCode():
    for row in crimeLocList:
        zip = row[3]
        print(zip)
        if not zip in crimePerZipDict:
            crimePerZipDict[zip] = 1
        else:
            crimePerZipDict[zip] += 1 
        
    with open('crime_zip_2016.csv', 'w') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['zip', 'Count_2016'])
        for key, value in crimePerZipDict.items():
            writer.writerow([k,v])

        
    

crimePerZipCode() will output a csv file with the number of felonies per zipcode.

In [ ]:
crimePerZipCode()